Create dataframes from Tables

In [0]:
df_customers = spark.read.table("ansira.customer")
df_customers.show()

df_items = spark.read.table("ansira.item")
df_items.show()

df_products = spark.read.table("ansira.product")
df_products.show()

df_orders = spark.read.table("ansira.order")
df_orders.show()

df_products = spark.read.table("ansira.product")
df_products.show()

# Querys
1.  List Florida customers 
 * Write a SQL query to return all customers in the state of "Florida" 
 * sorted in alphabetical order

In [0]:
import pyspark.sql.functions as F
df_customers_florida = (df_customers.filter(F.col("state") == "Florida")
                                    .orderBy(F.col("customer_id"))
)
df_customers_florida.show()

1. Count products by category 
- Write a SQL query to return the number of product under each category

In [0]:
df_count_category = (
    df_products.groupby(F.col("category"))
                .agg(F.count(F.col("product_id")).alias("count"))
                .orderBy(F.col("count").desc())
)    
df_count_category.show()

2. Count products by category 
- Write a SQL query to return the number of product under each category

In [0]:
df_count_category = (
    df_products.groupby(F.col("category"))
                .agg(F.count(F.col("product_id")).alias("count"))
                .orderBy(F.col("count"))
)
df_count_category.show()

3. Most expensive & cheapest products by category 
- Write a SQL query that returns the name and price of the most expensive product and 
- the cheapest product (in the same row) for each product category

In [0]:
df_expensive_cheap = (
    df_products.join(df_products.groupBy("category")
                                .agg(
                                        F.max("price").alias("max_price"), 
                                        F.min("price").alias("min_price")
                                    )
                                , "category"
                    )
                .groupBy("category", "max_price", "min_price")
                .agg(
                        F.first(F.when(F.col("price") == F.col("max_price"), F.col("name")), ignorenulls = True).alias("expensive_Product"),
                        F.first(F.when(F.col("price") == F.col("min_price"), F.col("name")), ignorenulls = True).alias("cheap_product")
                    )
)
df_expensive_cheap.select("category", "expensive_Product", "max_price", "cheap_product", "min_price").show()

Using window functions

In [0]:
from pyspark.sql.window import Window

# Definir la ventana por categoría
w = Window.partitionBy("category").orderBy(F.col("price").desc())
w_min = Window.partitionBy("category").orderBy(F.col("price").asc())

# Producto más caro por categoría
df_max = df_products.withColumn("rank", F.row_number().over(w)) \
                   .filter(F.col("rank") == 1) \
                   .select("category", 
                           F.col("name").alias("most_expensive_product"),
                           F.col("price").alias("max_price"))

# Producto más barato por categoría
df_min = df_products.withColumn("rank", F.row_number().over(w_min)) \
                   .filter(F.col("rank") == 1) \
                   .select("category", 
                           F.col("name").alias("cheapest_product"),
                           F.col("price").alias("min_price"))

# Unir ambos resultados
df_result = df_max.join(df_min, "category").orderBy("category")

df_result.show()


3. Most popular products 
- Write a SQL query to return the top 10 most popular products ordered by customers in the state of FL in 2018. 
- The popularity of a product is determined by the overall quantity of items ordered for each product. 
- The result should include the product id, product name and total quantity ordered from higher to lower

In [0]:
from pyspark.sql.functions import broadcast

# Aplicar filtros antes de join
df_orders_filtered = df_orders.filter(F.year("order_date") == 2018)
df_customers_filtered = df_customers.filter(F.col("state") == "Florida")

df_popular_products = (
    df_customers_filtered
        .join(df_orders_filtered, "customer_id")
        .join(df_items, "order_id")
        .join(broadcast(df_products), "product_id")   # broadcast productos (tabla pequeña)
        .groupBy("product_id", "name")
        .agg(F.sum("quantity").alias("total_quantity"))
        .orderBy(F.desc("total_quantity"))
        .limit(10)
)
df_popular_products.show()

4. 2018 sales vs 2019 sales 
- Write a SQL query to compare the total amount of sales of all products under category "Toys" during 2018 vs 2019 (in the same row).

In [0]:
df_sales2018 = (
    df_items.join(df_products, "product_id")
            .join(df_orders, "order_id")
            .filter(
                (F.year("order_date") == "2018") & (F.col("category") == "Toys")
            )
            .groupBy("category")
            .agg(F.sum(F.col("unique_price") * F.col("quantity")).alias("sales_2018"))
)

df_sales2019 = (
    df_items.join(df_products, "product_id")
            .join(df_orders, "order_id")
            .filter(
                (F.year("order_date") == "2019") & (F.col("category") == "Toys")
            )
            .groupBy("category")
            .agg(F.sum(F.col("unique_price") * F.col("quantity")).alias("sales_2019"))
)

# --- Cross Join (una fila con ambas métricas) ---
df_result = (
    df_sales2018.crossJoin(df_sales2019)
    .withColumn("diff_sales", F.col("sales_2019") - F.col("sales_2018"))
    .withColumn(
        "growth_percent",
        F.round(((F.col("sales_2019") - F.col("sales_2018")) * 100.0) / F.when(F.col("sales_2018") != 0, F.col("sales_2018")), 2)
    )
)

df_result.show()

Using PIVOT

In [0]:
df_sales = (
    df_orders.join(df_items, "order_id")
            .join(df_products, "product_id")
            .filter(F.col("category") == "Toys")
            .withColumn("year", F.year("order_date"))
            .groupBy("year")
            .agg(F.sum(F.col("quantity") * F.col("price")).alias("sales"))
)

df_pivot = (
    df_sales.groupBy()
            .pivot("year", [2018, 2019])
            .agg(F.first("sales"))
            .withColumnRenamed("2018", "sales_2018")
            .withColumnRenamed("2019", "sales_2019")
            .withColumn("diff_sales", F.col("sales_2019") - F.col("sales_2018"))
            .withColumn(
                "growth_percent",
                F.round(((F.col("sales_2019") - F.col("sales_2018")) * 100.0) / 
                        F.when(F.col("sales_2018") != 0, F.col("sales_2018")), 2)
            )
)
df_pivot.show()

A single query

In [0]:
df_sales = (
    df_orders.join(df_items, "order_id")
            .join(df_products, "product_id")
            .filter(F.col("category") == "Toys")
            .agg(
                F.sum(F.when(F.year("order_date") == 2018, F.col("quantity") * F.col("unique_price"))).alias("sales_2018"),
                F.sum(F.when(F.year("order_date") == 2019, F.col("quantity") * F.col("unique_price"))).alias("sales_2019")
            )
            .withColumn("diff_sales", F.col("sales_2019") - F.col("sales_2018"))
            .withColumn(
                "growth_percent",
                F.round(((F.col("sales_2019") - F.col("sales_2018")) * 100.0) /
                        F.when(F.col("sales_2018") != 0, F.col("sales_2018")), 2)
            )
)
df_sales.show()


6. Total sales by Category

In [0]:
df_sales = (
    df_items.join(df_products, "product_id")
            .withColumn("total", F.col("unique_price") * F.col("quantity"))
            .groupby("category")
            .agg(F.sum("total").alias("total_sales"))
)
df_sales.show()